In [38]:
import os
import numpy as np
import pandas as pd
import math
from math import e

import configparser

In [39]:
# establecer directorio de trabajo por defecto y paths
working_dir='../Tests/Test1/'
pathoutput = working_dir + 'output/'

In [40]:
## parametros globales
parser = configparser.ConfigParser()
parser.read(working_dir + 'input/params.txt')

## asigna los parámetros globales
HSCD = float(parser['CALCULOS']['horas_sobrecarga_dia'])
PVDT = float(parser['CALCULOS']['perdida_vida_diaria_teorica'])
TNTR = float(parser['CALCULOS']['temperatura_normal_trafo'])
TAMB = float(parser['CALCULOS']['temperatura_ambiente'])
PDRE = float(parser['CALCULOS']['probabilidad_deterioro_reubicacion'])

## calculos con parametros globales
HVUT = 24 / PVDT
AVUT = HVUT / (24*365) 

##
## Resoluciones 818 y 819
##
##    Transformadores monofasicos - Perdidas en vacio
##
P1FVAC1 = float(parser['RES818819']['Par_1f_vac_1'])
P1FVAC2 = float(parser['RES818819']['Par_1f_vac_2'])
              
##
##    Transformadores monofasicos - Perdidas con carga
##
P1FCAR1 = float(parser['RES818819']['Par_1f_car_1'])
P1FCAR2 = float(parser['RES818819']['Par_1f_car_2'])
P1FCAR3 = float(parser['RES818819']['Par_1f_car_3'])
P1FCAR4 = float(parser['RES818819']['Par_1f_car_4'])

##
##    Transformadores trifasicos -- Perdidas en vacio
##
P3FVAC11 = float(parser['RES818819']['Par_3f_vac_11'])
P3FVAC12 = float(parser['RES818819']['Par_3f_vac_12'])
P3FVAC21 = float(parser['RES818819']['Par_3f_vac_21'])
P3FVAC22 = float(parser['RES818819']['Par_3f_vac_22'])
P3FVAC31 = float(parser['RES818819']['Par_3f_vac_31'])
P3FVAC32 = float(parser['RES818819']['Par_3f_vac_32'])
        
##
##    Transformadores trifasicos - Perdidas con carga
##
P3FCAR11 = float(parser['RES818819']['Par_3f_car_11'])
P3FCAR12 = float(parser['RES818819']['Par_3f_car_12'])
P3FCAR13 = float(parser['RES818819']['Par_3f_car_13'])
P3FCAR21 = float(parser['RES818819']['Par_3f_car_21'])
P3FCAR22 = float(parser['RES818819']['Par_3f_car_22'])
P3FCAR31 = float(parser['RES818819']['Par_3f_car_31'])
P3FCAR32 = float(parser['RES818819']['Par_3f_car_32'])

##
## tablas de datos
##
        
nodos = pd.read_csv(working_dir + "input/nodos.csv", sep=',', decimal='.')
inv = pd.read_csv(working_dir + "input/inventario_transformadores.csv", sep=',', decimal='.')
cartectraf = pd.read_csv(working_dir + "input/carac_tecn_transf.csv", sep=',', decimal='.')
vidautil = pd.read_csv(working_dir + "input/vida_util.csv", sep=',', decimal='.')

## 
## adecuación de las tablas para facilidad en cálculos
##
inv = inv.merge(nodos[['id_nodo','id_nodo_Internexa']], on = 'id_nodo_Internexa',how = 'left')

In [41]:
# indicar a qué grupo pertenece el transformador, para el cálculo de las pérdidas de transformación
inv['grpt_t'] = 1
inv.loc[(inv.fases == 3) & (inv.capacidad_kVA >= 150), 'grpt_t'] = 2
inv.loc[(inv.fases == 3) & (inv.capacidad_kVA >= 800), 'grpt_t'] = 3

# indicar a qué grupo pertenece el transformador, para el cálculo de la pérdida de vida útil
inv['grpv_t'] = 1
inv.loc[(inv.fases == 1) & (inv.capacidad_kVA > 50), 'grpv_t'] = 2
inv.loc[(inv.fases == 3) & (inv.capacidad_kVA >= 150), 'grpv_t'] = 2
inv.loc[(inv.fases == 3) & (inv.capacidad_kVA >= 500), 'grpv_t'] = 3
                
# calcular la vida útil restante del transformador (en meses)
inv['vida_util_teorica_anos'] = AVUT
inv['vida_util_restante_meses'] = (inv.vida_util_teorica_anos - inv.anos_uso) * 12
inv.loc[inv.vida_util_restante_meses < 0, 'vida_util_restante_meses'] = 1

# indicar los grupos de la Vida Útil
vidautil['grpv_t'] = 1
vidautil.loc[(vidautil.fases == 1) & (vidautil.limite_inferior_potencia_KVA >= 50), 'grpv_t'] = 2
vidautil.loc[(vidautil.fases == 3) & (vidautil.limite_inferior_potencia_KVA >= 150),'grpv_t'] = 2
vidautil.loc[(vidautil.fases == 3) & (vidautil.limite_inferior_potencia_KVA >= 500), 'grpv_t'] = 3

# armar keys para búsquedas
cartectraf['faca'] = cartectraf.fases.map(str) + "-" + cartectraf.capacidad_kVA.map(str)
vidautil['tfcg'] = vidautil.T_gradC.map(str) + "-" + vidautil.fases.map(str) + "-" + vidautil.carga_precedente.map(str) + "-" + vidautil['grpv_t'].map(str)


In [42]:
##########################################
#  Definición de funciones #
##########################################

# obtener los parámetros un nodo
def parnodo(id_nodo):
    cmax_n = float(nodos[nodos['id_nodo']==id_nodo]['carga_maxima_kVA'])
    cpro_n = float(nodos[nodos['id_nodo']==id_nodo]['carga_promedio_kVA'])    
    pkwh_n = float(nodos[nodos['id_nodo']==id_nodo]['tarifa_KWh_usuario_predominante']) 
    return (cmax_n,cpro_n,pkwh_n)

# obtener los parámetros de un transformador
def partrafo(id_trafo):
    capa_t = float(inv[inv['id_trafo']==id_trafo]['capacidad_kVA'])
    fase_t = int(inv[inv['id_trafo']==id_trafo]['fases'])
    viut_t = int(inv[inv['id_trafo']==id_trafo]['vida_util_teorica_anos'])
    nodo_t = int(inv[inv['id_trafo']==id_trafo]['id_nodo'])
    creu_t = float(inv[inv['id_trafo']==id_trafo]['costo_actividad_reubicacion'])
    viur_t = float(inv[inv['id_trafo']==id_trafo]['vida_util_restante_meses'])
    grpt_t = int(inv[inv['id_trafo']==id_trafo]['grpt_t'])
    faca_t = str(fase_t) + '-' + str(capa_t)
    cnue_t = int(cartectraf[cartectraf['faca']==faca_t]['costo_a_nuevo'])
    return (capa_t,fase_t,viut_t,nodo_t,creu_t,viur_t,grpt_t,faca_t,cnue_t)

# calcular costo de las perdidas de transformacion de un par nodo*trafo
def cospertransf(id_nodo,id_trafo):
    if id_nodo == 999999:
        cpt_nt = 0
    else:
        # hallar parametros del nodo y del trafo
        cmax_n,cpro_n,pkwh_n = parnodo(id_nodo)
        capa_t,fase_t,viut_t,nodo_t,creu_t,viur_t,grpt_t,faca_t,cnue_t = partrafo(id_trafo)
        futi_nt = cmax_n / capa_t
            
        # calcular perdidas nominales en vacio y perdidas nominales con carga en funcion de las fases y el grupo en la fase
        if fase_t == 1:
            pnvac = P1FVAC1 * capa_t ** P1FVAC2
            pncar = P1FCAR1 * capa_t ** 3 + P1FCAR2 * capa_t ** 2 + P1FCAR3 * capa_t + P1FCAR4
        else:
            if grpt_t == 1:
                pnvac = P3FVAC11 * capa_t ** P3FVAC12
                pncar = P3FCAR11 * capa_t ** 2 + P3FCAR12 * capa_t + P3FCAR13
            if grpt_t == 2:
                pnvac = P3FVAC21 * capa_t ** P3FVAC22
                pncar = P3FCAR21 * capa_t + P3FCAR22
            if grpt_t == 3:
                pnvac = P3FVAC31 * capa_t ** P3FVAC32
                pncar = P3FCAR31 * capa_t + P3FCAR32
    
        # calcular las perdidas en hierro y cobre en unidades W 
        pfeW = pnvac
        pcuW = pncar * futi_nt ** 2
        ptrW = pfeW + pcuW
    
        # monetizacion de las perdidas
        cpt_nt = ptrW / 1000 * pkwh_n * 24 * 30 * viur_t
    return cpt_nt


# calcular costo de las perdidas de vida util de un par nodo*trafo
def costopervutil(id_nodo,id_trafo):
    if id_nodo == 999999:
        cpv_nt = 0
    else:
        # hallar parametros del nodo y del trafo
        cmax_n,cpro_n,pkwh_n = parnodo(id_nodo)
        capa_t,fase_t,viut_t,nodo_t,creu_t,viur_t,grpt_t,faca_t,cnue_t = partrafo(id_trafo)
    
        # calcular la carga precedente y el factor de utilizacion
        cpre_nt = cpro_n / capa_t
        futi_nt = cmax_n / capa_t
        
        # aproximar la carga precedente a los valores de la norma GTC50
        if cpre_nt < ((0.5 + 0.75) / 2):
            cpre_nt = 0.5
        else:
            if cpre_nt < ((0.75 + 0.9) / 2):
                cpre_nt = 0.75
            else:
                cpre_nt = 0.9
        
        # calcular el porcentaje diario de perdida de vida util real en porcentaje
        key = str(TAMB) + '-' + str(fase_t) + '-' + str(cpre_nt) + '-' + str(grpt_t)
        theta = temperPC(futi_nt,key)
        fevej = (HSCD / 24) * (e**(15000/383 - 15000/(theta + 273))-1)
        pvdr = PVDT * (1 + fevej)
    
        # valorar perdida de vida util restante en pesos, durante lo que queda de vida util del trafo en el nodo 
        cpv_nt = viur_t * 30 * cnue_t * pvdr
    return cpv_nt


# calcular la temperatura del punto mas caliente dado un factor de utilizacion
def temperPC(futi_nt,key):
    tabvu = vidautil.loc[(vidautil.tfcg == str(key)) & (vidautil.duracion_pico_carga_h <= HSCD)]
    ncargas = tabvu.shape[0]
    carga=futi_nt * 100
    theta=0
    if carga < tabvu['carga_porc'].min(): theta = TNTR
    if carga >= tabvu['carga_porc'].max(): theta = tabvu['max_temp_PC_gradC'].max()
    if theta == 0:
        tabvu = tabvu.sort_values(['carga_porc'],ascending=[False])
        for index, row in tabvu.iterrows():
            if carga <= row['carga_porc']:
                theta = row['max_temp_PC_gradC']
                break
    return theta 


# calcular los costos de la actividad de reubicacion
def costopermtrafo(id_trafo):
    cperm_t = float(inv[inv.id_trafo == id_trafo]['costo_actividad_reubicacion'])
    return cperm_t


# calcular los costos del deterioro por reubicacion
def costodetetrafo(id_trafo):
    cnue_t = partrafo(id_trafo)[8]
    cdete_t = cnue_t * PDRE
    return cdete_t


# encontrar dos parejas para hacer permutación. Una pareja es un par (nodo*trafo)
# si debe bloquear un nodo porque no encuentra pareja, debe repetir con el siguiente nodo mas costoso
def hallarparejasperm (sol):
    id_trafo2 = 0
    while (id_trafo2 ==0):       
        # identificar el nodo mas costoso y su trafo para intercambio
        id_nodo1, id_trafo1 = nodomascostoso (sol)            
        id_nodo2, id_trafo2 = trafomascostoso (id_nodo1,sol)
        
        # si no encuentra un trafo adecuado para el cambio, se bloquea el nodo
        if id_trafo2 == 0:
            sol.at[sol.id_nodo == id_nodo1,'bloq']=1
    return (id_nodo1, id_trafo1, id_nodo2, id_trafo2)


# elegir el nodo mas costoso aun no evaluado, junto con su trafo asociado
def nodomascostoso (sol):
    # seleccionar los nodos potenciales
    solcopia = sol.copy()
    solcopia = solcopia[(solcopia['eval'] == 0)]
    solcopia = solcopia[(solcopia['bloq'] == 0)]
    solcopia = solcopia[(solcopia['coper'] > 0)]
    
    # seleccionar el mas costoso de los nodos
    id_nodo1 = 0
    id_trafo1 = 0
    if solcopia.shape[0] > 0:
        solcopia = solcopia.sort_values(['coper'],ascending=[False])
        id_nodo1 = int(solcopia.iloc[0,0])
        id_trafo1 = int(solcopia.iloc[0,1])
    return (id_nodo1, id_trafo1)


# elegir el trafo mas costoso para hacer permutacion con un cierto nodo
def trafomascostoso (id_nodo1,sol):
    cmax_n1 = parnodo(id_nodo1)[0]
    id_trafo1 = int(sol[sol.id_nodo == id_nodo1]['id_trafo'])
    capa_t1 = partrafo(id_trafo1)[0]
    
    # seleccionar los transformadores potenciales
    solcopia = sol.copy()
    solcopia = solcopia[(solcopia.capacidad_kVA >= cmax_n1)]
    solcopia = solcopia[(solcopia.carga_maxima_kVA <= capa_t1)]
    solcopia = solcopia[(solcopia.perm == 0)]
    solcopia = solcopia[(solcopia.bloq == 0)]
    solcopia = solcopia[(solcopia.id_nodo != id_nodo1)]

    # restricción de cargabilidad minima de un trafo por auditoría de la CREG: 40%
    solcopia = solcopia[(solcopia.capacidad_kVA <= cmax_n1 / 0.4)]
    
    # seleccionar el mas costoso de los nodos de los trafos
    id_nodo2=0
    id_trafo2=0
    if solcopia.shape[0] > 0:
        solcopia = solcopia.sort_values(['coper'],ascending=[False])
        id_nodo2 = int(solcopia.iloc[0,0])
        id_trafo2 = int(solcopia.iloc[0,1])
    return (id_nodo2, id_trafo2)


# calcular los costos de permutacion de dos trafos en una solucion provisional respecto a una solucion original
def costopermpar(id_trafo1,id_trafo2,sol_orig,sol_prov):
    cperm_t1 = 0
    cperm_t2 = 0
    
    # para el trafo 1
    id_nodo1_orig = int(sol_orig[(sol_orig.id_trafo == id_trafo1)]['id_nodo'])
    id_nodo1_prov = int(sol_prov[(sol_prov.id_trafo == id_trafo1)]['id_nodo'])
    if id_nodo1_orig != id_nodo1_prov:
        cperm_t1 = costopermtrafo(id_trafo1)
        cdete_t1 = costodetetrafo(id_trafo1)
    
    # para el trafo 2
    id_nodo2_orig = int(sol_orig[(sol_orig.id_trafo == id_trafo2)]['id_nodo'])
    id_nodo2_prov = int(sol_prov[(sol_prov.id_trafo == id_trafo2)]['id_nodo'])
    if id_nodo2_orig != id_nodo2_prov:
        cperm_t2 = costopermtrafo(id_trafo2)
        cdete_t2 = costodetetrafo(id_trafo2)
    return (cperm_t1,cdete_t1,cperm_t2,cdete_t2)


#Verificar si se ha cumplido la condición de parada de las permutaciones
def condparada(sol):
    condicion_parada = 0
    #Seleccionar los nodos potenciales
    sol = sol.loc[(sol['eval'] == 0)]
    sol = sol.loc[(sol['bloq'] == 0)]
    sol = sol.loc[(sol['coper'] > 0)]
    if sol.shape[0] == 0: condicion_parada = 1
    return condicion_parada

#Función para cálculos en bodega
def bodega(sol_orig,sol_final):
    bod_orig = sol_orig[sol_orig.id_nodo == 999999]
    tf_bod_orig = bod_orig.shape[0]
    cp_bod_orig = bod_orig['capacidad_kVA'].sum()

    bod_final = sol_final[sol_final.id_nodo == 999999]
    tf_bod_final = bod_final.shape[0]
    cp_bod_final = bod_final['capacidad_kVA'].sum()

    return (bod_orig,bod_final)


In [43]:
# funcion para armar la solucion original con sus costos

def armarsolorig():
    sol_orig = pd.DataFrame(columns=['id_nodo','id_trafo','cpt','cvu','coper','cperm','cdete','eval','perm','bloq'])

    for index, row in inv.iterrows():
        id_nodo = int(row['id_nodo'])
        id_trafo = int(row['id_trafo'])

        # calculos de costos para los nodos reales y en bodega
        cpt_nt = 0
        cpv_nt = 0
        if id_nodo != 999999: 
            cpt_nt = cospertransf (id_nodo,id_trafo)
            cpv_nt = costopervutil (id_nodo,id_trafo)
        coper_nt = cpt_nt + cpv_nt              
        
        # almacenar el costo para ese arreglo en particular
        sol_orig.loc[index]= [id_nodo,id_trafo,cpt_nt,cpv_nt,coper_nt,0,0,0,0,0]
        
    # poner carga maxima de nodos y capacidades de transformadores en la solucion original
    sol_orig = sol_orig.merge(inv[['id_trafo','capacidad_kVA']], on = 'id_trafo',how = 'left')
    sol_orig = sol_orig.merge(nodos[['id_nodo','carga_maxima_kVA']], on = 'id_nodo',how = 'left')

    # escribir la solucion original y calcular su costo
    sol_orig.to_csv(pathoutput + 'solucion_original.csv')
    c_sol_orig = sol_orig['coper'].sum()
    
    return (sol_orig,c_sol_orig)

In [44]:
# funcion para hacer una permutacion

def permutar(sol):
    
    # elegir el nodo mas costoso y el trafo mas costoso que se le adecua
    id_nodo1, id_trafo1, id_nodo2, id_trafo2 = hallarparejasperm (sol)
    sol.at[sol.id_nodo == id_nodo1,'eval'] = 1

    # calcular los costos de operacion actuales en los dos nodos
    coperact_n1 = 0
    coperact_n2 = 0
    if id_nodo1 != 999999: coperact_n1 = float(sol[sol.id_nodo == id_nodo1]['coper'])
    if id_nodo2 != 999999: coperact_n2 = float(sol[sol.id_nodo == id_nodo2]['coper'])
    coperact = coperact_n1 + coperact_n2

    # calcular los costos de operacion despues de la permutacion en los dos nodos
    cpt_nt1 = cospertransf(id_nodo1,id_trafo2)
    cpv_nt1 = costopervutil (id_nodo1,id_trafo2)
    cpt_nt2 = cospertransf(id_nodo2,id_trafo1)
    cpv_nt2 = costopervutil (id_nodo2,id_trafo1)
    coperper_n1 = cpt_nt1 + cpv_nt1
    coperper_n2 = cpt_nt2 + cpv_nt2
    coperper = coperper_n1 + coperper_n2

    # calcular los costos de permutacion de los transformadores
    sol_prov = sol.copy()
    sol_prov.at[(sol.id_nodo == id_nodo1) & (sol.id_trafo == id_trafo1),'id_trafo']=id_trafo2
    sol_prov.at[(sol.id_nodo == id_nodo2) & (sol.id_trafo == id_trafo2),'id_trafo']=id_trafo1
    cperm_t1,cdete_t1,cperm_t2,cdete_t2 = costopermpar(id_trafo1,id_trafo2,sol_orig,sol_prov)
    cperm = cperm_t1 + cperm_t2
    cdeter = cdete_t1 + cdete_t2

    # hacer el cambio oficial en la solucion
    per_hecha = 0
    if coperact >= (coperper + cperm + cdeter):
        per_hecha = 1
        
        # traer las capacidades
        capa_t1 = partrafo(id_trafo1)[0]
        capa_t2 = partrafo(id_trafo2)[0]
            
        #Actualizar la matriz de solución
        sol.at[(sol.id_nodo == id_nodo1) & (sol.id_trafo == id_trafo1),'coper']=coperper_n1
        sol.at[(sol.id_nodo == id_nodo2) & (sol.id_trafo == id_trafo2),'coper']=coperper_n2
        sol.at[(sol.id_nodo == id_nodo1) & (sol.id_trafo == id_trafo1),'cperm']=cperm_t2
        sol.at[(sol.id_nodo == id_nodo2) & (sol.id_trafo == id_trafo2),'cperm']=cperm_t1
        sol.at[(sol.id_nodo == id_nodo1) & (sol.id_trafo == id_trafo1),'cdete']=cdete_t2
        sol.at[(sol.id_nodo == id_nodo2) & (sol.id_trafo == id_trafo2),'cdete']=cdete_t1
        sol.at[(sol.id_nodo == id_nodo1) & (sol.id_trafo == id_trafo1),'capacidad_kVA']=capa_t2
        sol.at[(sol.id_nodo == id_nodo2) & (sol.id_trafo == id_trafo2),'capacidad_kVA']=capa_t1
        sol.at[(sol.id_nodo == id_nodo1) & (sol.id_trafo == id_trafo1),'perm']=1
        sol.at[(sol.id_nodo == id_nodo2) & (sol.id_trafo == id_trafo2),'perm']=1
        sol.at[(sol.id_nodo == id_nodo1) & (sol.id_trafo == id_trafo1),'id_trafo']=id_trafo2
        sol.at[(sol.id_nodo == id_nodo2) & (sol.id_trafo == id_trafo2),'id_trafo']=id_trafo1        
        
    #Guardar estadísticas
    #sol.to_csv('solucion_final.csv')
    coper_sol = sol['coper'].sum()
    cperm_sol = sol['cperm'].sum()
    cdete_sol = sol['cdete'].sum()
    c_sol = coper_sol + cperm_sol + cdete_sol

    return (sol,coper_sol,cperm_sol,cdete_sol,c_sol,per_hecha)

In [45]:
########################################
######  Main #######
########################################

# calcular los costos de la solucion actual
sol_orig,c_sol_orig = armarsolorig()
sol = sol_orig.copy()

# armar la matriz de progresos en el hallazgo de mejores soluciones
prog_sol = pd.DataFrame(columns=['iteracion','tperm','coper','cperm','cdete','costo_total'])
prog_sol.loc[0] = [0,0,c_sol_orig,0,0,c_sol_orig]

# inicializar contadores y condicion de parada
titer = 0
tperm = 0
stop = condparada(sol)

# iterar
while (stop == 0):
    sol,coper_sol,cperm_sol,cdete_sol,c_sol,per_hecha = permutar(sol)
    tperm = tperm + per_hecha
    titer = titer + 1
    
    # guardar progreso de la simulacion
    prog_sol.loc[titer] = [titer,tperm,coper_sol,cperm_sol,cdete_sol,c_sol]     

    # evaluar condicion de parada
    stop = condparada(sol)

# calculos de la solucion final
sol_final = sol
bod_orig,bod_final = bodega(sol_orig, sol_final)

#Exportar los resultados
sol.to_csv(pathoutput + 'solucion_final.csv')
prog_sol.to_csv(pathoutput + 'progreso_soluciones.csv')
bod_orig.to_csv(pathoutput + 'bodega_original.csv')
bod_final.to_csv(pathoutput + 'bodega_final.csv')
print(c_sol_orig)
print(c_sol)

2555496087.6326895
1849098112.874599


In [ ]:
id_nodo = 1
carga_max_nodo = float(nodos[nodos.id_nodo==id_nodo]['carga_maxima_kVA'])
print(carga_max_nodo)

carga_max_nodo = float(nodos.carga_maxima_kVA[nodos.id_nodo==id_nodo])
print(carga_max_nodo)

carga_max_nodo = nodos.carga_maxima_kVA[nodos.id_nodo==id_nodo]
print(carga_max_nodo)